# Developing Apps

The Nesis' API gives you the ability to develop apps (new apps or extend existing apps) that interact with your private document and data repositories. Using Nesis' security model, you are able to apply role based access control to your data such that users are only able to seamlessly access data tied to their security profile.

Some use cases for the Nesis API.
1. You already have a mobile banking app and want to give it generative AI capabilities to help you customers to interract with their bank statements from within your mobile banking app.
2. You have an existing internal legacy customer service application that you'd like to extend and give AI capabilities.
3. You want to add generative AI powered conversation interraction to your website so that your users have tailored responses based on their personalized data.


## Create a Nesis App
In the Nesis frontend,

Navigate to **Settings** -> **Roles** and create a role with these properties
1. Name: `nesis-admin-app`
2. Policy:

    ```
    {
        "items": [
            {
                "action": "create",
                "resource": "users/*"
            },
            {
                "action": "update",
                "resource": "users/*"
            },
            {
                "action": "update",
                "resource": "roles/*"
            },
            {
                "action": "create",
                "resource": "roles/*"
            },
            {
                "action": "update",
                "resource": "datasources/*"
            },
            {
                "action": "create",
                "resource": "datasources/*"
            }
        ]
    }
    ```

Navigate to **Settings** -> **Apps** and then create an app with the following details
1. Name: `nesis-admin-app`
2. Description: `Nesis admin application`
3. Enabled: `Checked`
4. Role: `nesis-admin-app`

Note down the API key generated


Navigate to **Settings** -> **Apps** and then create another app with the following details
1. Name: `nesis-prediction-app`
2. Description: `Nesis admin application`
3. Enabled: `Checked`
4. Role: Do not select a role

Note down the API key generated

> This is all you need to do in the UI. The rest is all API driven using the API keys.

## Create a datasource

In [ ]:
import json
import requests

nesis_api_endpoint = "http://nesis-endpoint.com/v1"
admin_api_key = "the-admin-api-key"
predictions_api_key = "the-prediction-app-api-key"

In [9]:
admin_api_headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {admin_api_key}'}

In [11]:
prediction_api_header = {'Content-Type': 'application/json', 'Authorization': f'Bearer {predictions_api_key}'}

In [20]:
datasource = {"type": "minio", "name": "documents", "connection": {"user": "your_username","password":"your_password","endpoint": "http://minio:port","dataobjects": "bucket-name"}}

In [21]:
datasource_response = requests.post(f'{nesis_api_endpoint}/datasources', headers=admin_api_headers, json=datasource)

In [23]:
datasource = json.loads(datasource_response.text)

## Create a role to be used to query the datasource

In [24]:
role = {
    "name": "data-reader",
    "policy": {
      "items": [
        {
          "action": "create",
          "resource": "predictions/*"
        },
        {
          "action": "read",
          "resource": "datasources/*"
        }
      ]
    }
}


In [25]:
role_response = requests.post(f'{nesis_api_endpoint}/roles', headers=admin_api_headers, json=role)

In [26]:
print(role_response.text)

{"create_date":"2024-05-24 18:45:28","id":"f0e63ee3-d1af-4d56-9b99-e28bf308db45","name":"data-reader"}



In [27]:
role = json.loads(role_response.text)

## Create a user and assign the above role

In [28]:
user_response = requests.post(f'{nesis_api_endpoint}/users', headers=admin_api_headers, json={'email': 'some.email@domain.com', 'name': 'Da Zone', 'password': 'P@ssword', 'roles': [role['id']]})

In [29]:
print(user_response.text)

{"attributes":null,"create_date":"2024-05-24 18:47:01","email":"some.email@domain.com","enabled":true,"id":"b919c24d-7574-43ec-9889-200584e44d6b","name":"Da Zone","root":false}



In [ ]:
user = json.loads(user_response.text)

## Query using the prediction API key. 

> This fails because no role was attached to the prediction app.

In [30]:
prediction_response = requests.post(f'{nesis_api_endpoint}/modules/qanda/predictions', headers=prediction_api_header, json={'query': 'summarize the only office agreement'})

In [31]:
print(prediction_response)

<Response [403]>


## Query using the prediction API key and user header
> This succeeds because the prediction app inherits permissions from the user (who has the right permission) using the `X-Nesis-Request-UserKey` header.

In [40]:
prediction_response = requests.post(f'{nesis_api_endpoint}/modules/qanda/predictions', headers={**prediction_api_header, 'X-Nesis-Request-UserKey': user['id']}, json={'query': 'summarize the only office agreement'})

In [41]:
print(prediction_response.text)

{"create_date":"2024-05-24 18:59:17","data":{"choices":[{"delta":null,"finish_reason":"stop","index":0,"message":{"content":"The Only Office Agreement is a legal document that outlines the terms and conditions of leasing office space. It typically includes details such as the duration of the lease, rental payments, maintenance responsibilities, and any other specific provisions agreed upon by the landlord and tenant. This agreement serves to protect the rights and obligations of both parties involved in the leasing of office space.","role":"assistant"},"sources":[]}],"created":1716577157,"id":"a0bc9a93-a3e5-45e8-9d4f-090f652e8dc2","model":"rag","object":"completion"},"id":null,"input":"summarize the only office agreement","module":"qanda"}



## Conclusion
This notebook shows you a simplistic workflow of how to programatically interract with Nesis. We see how to create an app and assign it **CREATE** permissions on **USERS**, **DATASOURCES** and **ROLES**. With this app, we are able to programmatically create multiple users, datasources and roles.

By doing this you are able to integrate Nesis programically into your existing worklows, automating the creation of Nesis users and roles inline with your existing enterprise user management system and do this at scale.

Lastly, we see how Nesis can be easily integrated into your existing application such as a website and easily tailored responses to your logged in website users, thus giving your website generative AI capabilities.

Next steps;

1. View the Nesis repository https://github.com/ametnes/nesis/
2. View the documentation https://ametnes.github.io/nesis/.
3. Join our [Slack community](https://join.slack.com/t/nesiscommunity/shared_invite/zt-2gpp38ts2-tAfea6R_q9RHudhPEcBJSA) and let us know how you get on or ask any questions.